## Project Template (Deepnote)

You can use this notebook as a starting point for your class project and/or the extra exercise for graduate students in Deepnote. It comes set up with the dependencies we use in class, so you don't have to install these yourself.

**Notes:**
- To use this in your own workspace, click “Duplicate” for the project in Deepnote.
- Class dependencies, namely `drake` and the `manipulation` package, are already preinstalled in the Deepnote image for this course, as well as a bunch of other dependencies that these packages depend on or that we use in class
- To see the full list of installed dependencies, as well as version numbers etc, see this file: [pyproject.toml](https://github.com/RussTedrake/manipulation/blob/master/pyproject.toml).
  - **Note**: we install all the extra dependencies, including the `dev` dependencies, into the Docker image that this deepnote project runs on.


In [ ]:
%load_ext autoreload
%autoreload 2
# ^ make it so we can change inner files


In [2]:

import numpy as np
from pydrake.all import (
    ContactModel,
    DiagramBuilder,
    MeshcatVisualizer,
    MultibodyPlant,
    Rgba,
    RigidTransform,
    RollPitchYaw,
    SceneGraph,
    Simulator,
    Sphere,
    StartMeshcat,
    VectorLogSink,
    VisualizationConfig,
)

from manipulation.meshcat_utils import AddMeshcatTriad
from manipulation.station import (
    AddPointClouds,
    MakeHardwareStation,
    Parser,
)
from scripts.load_scenario import load_scenario

In [3]:
# Start meshcat for visualization
meshcat = StartMeshcat()
print("Click the link above to open Meshcat in your browser!")

INFO:drake:Meshcat listening for connections at http://localhost:7000


Click the link above to open Meshcat in your browser!


In [4]:
use_hydroelastic = False # this takes forever

def make_builder():
    scenario = load_scenario()

    # since makehardwarestation finalizes the plant before giving it back to us
    def callback(parser: Parser):
        plant = parser.plant()
        plant.set_contact_model(ContactModel.kHydroelasticWithFallback)
        scene_graph: SceneGraph = parser.scene_graph()
        config = scene_graph.get_config()
        if use_hydroelastic:
            config.default_proximity_properties.compliance_type = "compliant"
        scene_graph.set_config(config)

    station = MakeHardwareStation(scenario, meshcat=meshcat, parser_prefinalize_callback=callback)
    plant: MultibodyPlant = station.plant()


    builder = DiagramBuilder()
    builder.AddSystem(station)

    to_point_cloud = AddPointClouds(
        scenario=scenario, station=station, builder=builder, meshcat=meshcat
    )
    builder.ExportOutput(
        to_point_cloud["camera0"].point_cloud_output_port(), "point_cloud0"
    )

    return builder, plant, station

# diagram = builder.Build()


In [6]:
from scripts.grasp_broom import plan_path, build_temp_plant, combine_trajectory

# basic test
# X_WG1 = RigidTransform(RollPitchYaw(np.pi/2, 0, 0), [0, 0.5, 1])
# X_WG2 = RigidTransform(RollPitchYaw(np.pi/2, 0, np.pi/2), [0, 1, 1])

# meshcat.SetObject("start", Sphere(0.02), rgba=Rgba(0.1, 0.9, 0.1, 1))
# meshcat.SetTransform("start", X_WG1)
# meshcat.SetObject("goal", Sphere(0.02), rgba=Rgba(0.1, 0.9, 0.1, 1))
# meshcat.SetTransform("goal", X_WG2)


In [5]:
# test function to see the temp plant

# diagram, plant, gripper_frame = build_temp_plant(q0 = None, meshcat = meshcat)
# config = VisualizationConfig(lcm_bus='default', publish_period=0.015625, publish_illustration=True, default_illustration_color=Rgba(r=0.9, g=0.9, b=0.9, a=1.0), publish_proximity=True, default_proximity_color=Rgba(r=0.8, g=0.0, b=0.0, a=1.0), initial_proximity_alpha=0.5, publish_contacts=True, publish_inertia=True, enable_meshcat_creation=True, delete_on_initialization_event=True, enable_alpha_sliders=False)
# context = diagram.CreateDefaultContext()
# diagram.ForcedPublish(context)
# meshcat.SetObject("gripper", Sphere(0.02), rgba=Rgba(0.1, 0.9, 0.1, 1))
# meshcat.SetTransform("gripper", gripper_frame.GetPoseInWorld(context))


In [ ]:
# path = plan_path(X_WG1, X_WG2)

In [15]:
from pydrake.systems.primitives import TrajectorySource, ConstantVectorSource
from scripts.ik import solve_ik_for_pose

builder, plant, station = make_builder()

# grasp broom
X_WStart = RigidTransform(RollPitchYaw(np.pi, 0, 0), [0, 1.0, 0.5])
X_WMid = RigidTransform(RollPitchYaw(np.pi, 0, 0), [0.3, 1.8, 0.5])
X_WGoal = RigidTransform(RollPitchYaw(np.pi, 0, 0), [0.6, 1.8, 0.5])

AddMeshcatTriad(meshcat, 'a', X_PT=X_WStart)
AddMeshcatTriad(meshcat, 'b', X_PT=X_WMid)
AddMeshcatTriad(meshcat, 'c', X_PT=X_WGoal)

# traj1 = plan_path(X_WStart, X_WMid)
# traj2 = plan_path(X_WMid, X_WGoal)
# traj = combine_trajectory(traj1, traj2)
traj = plan_path(X_WStart, X_WGoal)

# print 20 joint positions sampled
times = np.linspace(traj.start_time(), traj.end_time(), 20)
for t in times:
    print(f'time{t}: {traj.value(t).flatten()}')

# add traj to builder and connect
q_src = builder.AddSystem(TrajectorySource(traj))
builder.Connect(q_src.get_output_port(), station.GetInputPort("iiwa.position"))
wsg_src = builder.AddSystem(ConstantVectorSource([1.0]))
builder.Connect(wsg_src.get_output_port(), station.GetInputPort("wsg.position"))

diagram = builder.Build()


7
(7, 10)
Collision optimization failed
time0.0: [-0.60453133  0.91819885 -1.12117104 -1.87791649 -1.39840897  1.59578909
 -1.74933769]
time0.18421052631578946: [-0.5031797   0.82521174 -1.0421649  -1.75259308 -1.27363629  1.60436472
 -1.64571197]
time0.3684210526315789: [-0.25804352  0.61535523 -0.84084441 -1.47291022 -0.99415536  1.55815344
 -1.42447428]
time0.5526315789473684: [ 0.04274124  0.39187896 -0.57069412 -1.18276995 -0.70152149  1.3505549
 -1.21923025]
time0.7368421052631579: [ 0.33213546  0.22251493 -0.27976832 -0.97998518 -0.47627719  0.98883042
 -1.08921487]
time0.9210526315789473: [ 0.58026908  0.11241897 -0.00655412 -0.88116728 -0.29147033  0.68084706
 -0.96263422]
time1.1052631578947367: [ 0.76303094  0.05966359  0.21242494 -0.8924266  -0.11200198  0.64260901
 -0.75974565]
time1.2894736842105263: [ 0.88948974  0.05033553  0.35726891 -0.98232612  0.05027599  0.88380783
 -0.49132304]
time1.4736842105263157: [ 0.99624574  0.0605761   0.42187159 -1.08827321  0.14455342  1

In [16]:

simulator = Simulator(diagram)
simulator.set_target_realtime_rate(1.0)
meshcat.StartRecording()
simulator.Initialize()
simulator.AdvanceTo(traj.end_time())
meshcat.StopRecording()
meshcat.PublishRecording()